In [1]:
import boto3
from double_click import echo

from nab3 import AWS as NabAWS,  Filter
from nab3.helpers.cloud_watch import md_statistics_summary, set_n_service_stats

AWS = NabAWS(boto3.Session())
"""
Passing boto3.Session here to illustrate how multiple sessions are supported.
The Service's Session is tied to the nab3.AWS instance that was used to create it.

So, if you wanted to retrieve all ASGs in eu-central-1 and us-east-1 

US_AWS = NabAWS(boto3.Session(region_name='us-east-1'))
EU_AWS = NabAWS(boto3.Session(region_name='eu-central-1'))

yeah you could get cute with this but it's just an example
us_asgs = await US_AWS.asg.list()
eu_asgs = await EU_AWS.asg.list()

"""

In [ ]:
    
ASG = await AWS.asg.list()
""" 
Alternatively, you could set ASG = await AWS.ecs_cluster.list() and the blocks below would behave identically. 
This is a demonstration of the flexibility of nab3 as well as it's consistency to allow for greater code reuse. 

# ASG = await AWS.ecs_cluster.list()
"""

In [2]:
# Include all ASGs where the name contains either of the provided strings in the list
f = Filter(name__icontains_any=['sample_service_1', 'sample_service_2'])
filtered_asg = await f.run(ASG)
print([c.name for c in filtered_asg])

['sample_service_1', 'sample_service_2']


In [9]:
await set_n_service_stats(filtered_asg)

In [10]:
md_output = ""
for asg in filtered_asg:
    md_output += f"#{asg.name}\n"
    
    for stats in asg.stats_list:
        md_output += md_statistics_summary(stats, False)

    md_output += "\n---\n"

with open('asg_stats.md', 'w') as f:
    f.write(md_output)

In [ ]:
cluster = filtered_asg[0]

# fields will return a dict containing the instance's attributes along with the attribute's type.
echo(cluster.fields())

# instances hasn't been explicitly loaded so this will return False
echo(cluster.instances.is_loaded())
await filtered_asg.fetch('instances')

# Now it works
instance = cluster.instances[0]
echo(f'Instances loaded {cluster.instances.is_loaded()}\n{instance.fields()}')

